# Overview

The purpose of this notebook is to demonstrate display and inspection of satellite imagery.

# Setup

Colab does not contain the [google/earthengine-jupyter](https://github.com/google/earthengine-jupyter) package, so run a shell command to install it.

In [ ]:
!pip install -q git+https://github.com/google/earthengine-jupyter.git@704c5f31a7d2de855e90357320d94425a7b6a791

  Preparing metadata (setup.py) ... done


Import packages used in this example.

In [ ]:
import datetime  # for date object manipulation
import ee  # the Earth Engine Python client library
import google  # for authentication to Google services
import ipywidgets as widgets  # interactive widgets (standard)
import ipyleaflet  # interactive mapping widget
from ee_jupyter.layout import MapWithInspector 

Specify the name of a [Google Cloud Project](https://console.cloud.google.com/) that will be used for data requests to Earth Engine. This project needs to have the [Google Earth Engine API](https://console.cloud.google.com/apis/api/earthengine.googleapis.com/) enabled. 

In [ ]:
COLAB_AUTH_FLOW_CLOUD_PROJECT_FOR_API_CALLS = 'tylerickson-misc'

Authenticate access to Earth Engine, using the Colab authentication workflow.

In [ ]:
# Authenticate to populate Application Default Credentials in the Colab VM.
google.colab.auth.authenticate_user()
# Create credentials needed for accessing Earth Engine.
credentials, auth_project_id = google.auth.default()
# Initialize Earth Engine.
ee.Initialize(credentials, project=COLAB_AUTH_FLOW_CLOUD_PROJECT_FOR_API_CALLS)
print('\N{check mark} Successfully initialized!')

✓ Successfully initialized!


# Analysis: Inspection of Earth Data

This section demonstrates how to display Earth data on an interactive map.

First, define a set of datasets that will be made available for visualization. Additional datasets can be found in the [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets).

In [ ]:
datasets = {
    'Sentinel-2': {
        'desc': 'Optical imagery: Sentinel-2 TOA, 2015-present',
        'base_collection': "COPERNICUS/S2_HARMONIZED",
        'vis_params': {
          'bands': ['B4', 'B3', 'B2'],
          'min': 0,
          'max': 3000,
        }
    },
    'Dynamic World V1':{
        'desc': 'Land Cover / Land Use Probabilities: Based on Sentinel-2, 2015-2100',
        'base_collection': "GOOGLE/DYNAMICWORLD/V1",
        'vis_params': {
          'bands': 'label',
          'min': 0,
          'max': 8,
          'palette': ['419BDF', '397D49', '88B053', '7A87C6',
                      'E49635', 'DFC35A', 'C4281B', 'A59B8F', 'B39FE1']
        }
    },
    'MOD14A1.061': {
        'desc': 'Thermal Anomalies & Fire (MODIS) 2000-present',
        'base_collection': "MODIS/061/MOD14A1",
        'vis_params': {
          'bands': ['MaxFRP', 'FireMask', 'FireMask'],
          'min': 0,
          'max': 6000,
        }
    },
    'NEX-GDDP tas':{
        'desc': 'Climate projection (tas; GFDL-ESM4; Scenario SSP245) 2015-2100',
        'base_collection': "NASA/GDDP-CMIP6",
        'filter': ee.Filter.And(
            ee.Filter.eq('model', 'GFDL-ESM4'),
            ee.Filter.eq('scenario', 'ssp245'),
        ),
        'vis_params': {
          'bands': ['tas'], # Daily near-surface air temperature.
          'min': 273.15 - 25,
          'max': 273.15 + 50,
          'palette': ['white', 'grey', 'pink', 'red']
        }
    },
    'NEX-GDDP pr':{
        'desc': 'Climate projection (pr; GFDL-ESM4; Scenario SSP245) 2015-2100',
        'base_collection': "NASA/GDDP-CMIP6",
        'filter': ee.Filter.And(
            ee.Filter.eq('model', 'GFDL-ESM4'),
            ee.Filter.eq('scenario', 'ssp245'),
        ),
        'vis_params': {
          'bands': ['pr'], # Daily near-surface air temperature.
          'min': 0,
          'max': 5e-4,
          'palette': ['white', 'blue']
        }
    },
}

Define widgets that allow a user to select a dataset, starting date and time interval.

In [ ]:
dataset_picker = widgets.Dropdown(
    options=[(v['desc'], k) for k,v in datasets.items()],
    description='Dataset:',
    layout=widgets.Layout(width='600px')
)

date_picker = widgets.DatePicker(
    description='Start Date:',
    value=datetime.date(2023, 4, 28)
)

num_days_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=10,
    step=1,
    description='# Days:',
    continuous_update=False
)

Define a function to update the data layers displayed on the map. This function is called when the widget controls are updated.

In [ ]:
def update_map_layer(_map):
  if len(_map.layers) > baselayer_count(_map):
    # Remove the tile layer.
    _map.remove_layer(_map.layers[baselayer_count(_map)])

  if date_picker.value and num_days_slider.value:

    dataset = datasets[dataset_picker.value]

    # Select a collection of satellite images from a date window.
    start_date = date_picker.value
    end_date = date_picker.value + datetime.timedelta(days=num_days_slider.value)
    collection = (
        ee.ImageCollection(dataset['base_collection'])
          .filterDate(
              start_date.isoformat(),
              end_date.isoformat()
          )
    )
    if 'filter' in dataset:
      collection = collection.filter(dataset['filter'])

    # Add a new tile layer.
    _map.addLayer(
      eeObject=collection.mosaic(),
      visParams=dataset['vis_params'],
      name=dataset_picker.value
    )

Define a set of [basemap tile layers](https://ipyleaflet.readthedocs.io/en/latest/map_and_basemaps/basemaps.html) to display on the interactive map.

In [ ]:
positron = ipyleaflet.basemap_to_tiles(ipyleaflet.basemaps.CartoDB.Positron)
positron.base = True
positron.name = 'Positron Layer'

satellite = ipyleaflet.basemap_to_tiles(ipyleaflet.basemaps.Esri.WorldImagery)
satellite.base = True
satellite.name = 'Satellite Layer'

def baselayer_count(map):
  return len([x for x in map.layers if x.base])

Trigger updating the map when a widget is updated.

In [ ]:
def handle_dataset_change(change, **kwargs):
  update_map_layer(map1.map)

dataset_picker.observe(handle_dataset_change, names=['value'])

def handle_date_change(change, **kwargs):
  update_map_layer(map1.map)

date_picker.observe(handle_date_change, names=['value'])

def handle_num_days_change(change, **kwargs):
  update_map_layer(map1.map)

num_days_slider.observe(handle_num_days_change, names=['value'])

Display the interactive map and widgets.

In [ ]:
map1 = MapWithInspector(
    center=(48.895557, 2.388747), zoom=7,
    layers=[satellite, positron]
)
update_map_layer(map1.map)
widgets.VBox([
    dataset_picker,
    widgets.HBox([date_picker, num_days_slider]),
    map1
])

## Exploring the Data

Explore the data by doing the following:

*   Pan/Zoom the map.
*   Change the dataset selector.
*   Change the start date and/or time interval.

If you find something interesting, you can use the following code block to print code that programmatically sets the widget controls.

In [ ]:
print(f'map1.map.center = {map1.map.center}')
print(f'map1.map.zoom = {map1.map.zoom}')
print(f"dataset_picker.value = '{dataset_picker.value}'")
print(f"date_picker.value = datetime.date({date_picker.value.year}, "
                                        f"{date_picker.value.month}, "
                                        f"{date_picker.value.day})")
print(f'num_days_slider.value = {num_days_slider.value}')

map1.map.center = [48.895557, 2.388747]
map1.map.zoom = 7.0
dataset_picker.value = 'Sentinel-2'
date_picker.value = datetime.date(2023, 4, 28)
num_days_slider.value = 1


## Examples

### Recent fire in the Brazilian Amazon


In [ ]:
# Center map on Cité des sciences et de l'industrie, Paris.
map1.map.center = [-11.58423973562834, -52.50580264266333]
map1.map.zoom = 11.0
dataset_picker.value = 'MOD14A1.061'
date_picker.value = datetime.date(2023, 4, 28)
num_days_slider.value = 10

### Projected daily surface temperature for 2050 (a single simulation)

In [ ]:
map1.map.center = [14.202428165137302, 64.63615416897834]
map1.map.zoom = 2.0
dataset_picker.value = 'NEX-GDDP tas'
date_picker.value = datetime.date(2050, 7, 1)
num_days_slider.value = 5

### Cité des sciences et de l'industrie, Paris, peeking through the clouds

In [ ]:
# Center map on Cité des sciences et de l'industrie, Paris.
map1.map.center = (48.895557, 2.388747)
map1.map.zoom = 11
dataset_picker.value = 'Sentinel-2'
date_picker.value = datetime.date(2023, 4, 28)
num_days_slider.value = 1